# Task 4
## Assignment for PhD positions in computational social sciences here at ETH Züric

### by Marius Herget




#### Step 1
Import all necessary packages and datasets. (including creationg of some helper functions)


In [4]:
import pandas as pd
import scipy as scipy
import networkx as nx
import matplotlib.pyplot as plt
import requests

In [5]:
# Helper functions
def normalize(df, column_name):
    result = df.copy()
    max_value = df[column_name].max()
    min_value = df[column_name].min()
    result[column_name+"_normalized"] = (df[column_name] - min_value) / (max_value - min_value)
    return result
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x.to_markdown())
    pd.reset_option('display.max_rows')

In [6]:
# Import data provided by github
adjax_matices = pd.read_csv("./task4_data/adjacency_matrices-115.csv",sep="\t", index_col=0, header=0)
adjax_matices.columns = adjax_matices.columns.astype('int64')
bills = pd.read_csv("./task4_data/bills-115.csv",sep="\t",header=0,index_col="bill_id")
edgelists = pd.read_csv("./task4_data/edgelists-115.csv",sep="\t",header=0,index_col=0)
individuals = pd.read_csv("./task4_data/individuals.csv",sep="\t",header=0,index_col="SGID")
members = pd.read_csv("./task4_data/members-115.csv",sep="\t",header=0,index_col="SGID")

##### *bills_in_better* 

This is a better version of the given _bills_ dataset: it was created by the code in step 4 and afterward saved in a file.


In [7]:
# Import extra data from generated data
bills_in_better = pd.read_csv("./extraData/better_bills.csv")

##### *individuals_money* 

This is a manipulated version of the given _individuals_ dataset: it was created by the code in step 3, modified in step 4, and afterward saved in a file to avoid unnecessary traffic.

In [8]:
# Import extra data from generated data
individuals_money = pd.read_csv("./extraData/individuals_money.csv")

#### Step 2
Do some basic calculations on the dataset to understand it.

In [9]:
# Basic analysis to understand the dataset
summedupMatrix_gived_signatures = adjax_matices.sum(axis=1,)
summedupMatrix_received_signatures = adjax_matices.sum(axis=0,)
index = summedupMatrix_received_signatures.idxmax()
 
print('The higehst number of given cosponsorships:')
print(summedupMatrix_gived_signatures.loc[index])
print('The higehst number of received cosponsorships:')
print(summedupMatrix_received_signatures.loc[index])

The higehst number of given cosponsorships:
674
The higehst number of received cosponsorships:
1689


#### Step 3
##### Importance of each representative


As mentioned in my submission to calculate the importance of each representative, available/spent money is a crucial factor. Another factor is the popularity within the voting base, experience, and standing within the own party. Therefore, re-election is also considered within the evaluation.

This data is affected by step 4 as well. Hence we will load it from a csv as well (saving in step 4).

##### *follow_the_money*

This data is needed to calculate the importance of each representative.

Reference: *House Office Expenditure Data, propublica.org, visited: 24.02.2020, URL: https://projects.propublica.org/represent/expenditures*

In [10]:
# Import extra from Congress 115 Money Spending
def load_follow_the_money():
    follow_the_money = pd.read_csv("./extraData/house-office-expenditures-with-readme/2017Q1-house-disburse-summary.csv")
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2017Q2-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2017Q3-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2017Q4-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2018Q1-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2018Q2-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2018Q3-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2018Q4-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2019Q1-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2019Q2-house-disburse-summary.csv"), ignore_index = True, sort=False)
    follow_the_money = follow_the_money.append(pd.read_csv("./extraData/house-office-expenditures-with-readme/2019Q3-house-disburse-summary.csv"), ignore_index = True, sort=False)
    return follow_the_money

In [11]:
def calculateImportanceRepresentative():
    # Representative importance
    follow_the_money = load_follow_the_money()
    follow_the_money["BIOGUIDE_ID"] = follow_the_money["BIOGUIDE_ID"].astype(str) 
    follow_the_money = follow_the_money.drop(columns="YEAR")
    follow_the_money = follow_the_money.drop(columns="YTD")
    follow_the_money = follow_the_money.groupby(['BIOGUIDE_ID']).sum()
    individuals_money = pd.merge(individuals,follow_the_money,left_on=["bioguide_id"], right_on = ['BIOGUIDE_ID'], how = 'left')
    # individuals_money => mapped data of representatives with money spending from 2017 Q1 to 2019 Q3
    individuals_money = normalize(individuals_money, "AMOUNT")
    individuals_money = normalize(individuals_money, "115_signatures_received")
    individuals_money = normalize(individuals_money, "115_signatures_given")

    # Consideration of re-elections
    individuals_money.loc[individuals_money["111"] == True, "in_congress_last_five_periods"] = 1
    individuals_money.loc[individuals_money["111"] == False, "in_congress_last_five_periods"] = 0
    individuals_money.loc[individuals_money["112"] == True, "in_congress_last_five_periods"] += 1
    individuals_money.loc[individuals_money["113"] == True, "in_congress_last_five_periods"] += 1
    individuals_money.loc[individuals_money["114"] == True, "in_congress_last_five_periods"] += 1
    individuals_money["in_congress_last_five_periods"] = individuals_money["in_congress_last_five_periods"]/4

    individuals_money["importance"] = individuals_money["AMOUNT_normalized"] * 0.5 
    individuals_money["importance"] += individuals_money["in_congress_last_five_periods"] * 0.5
    individuals_money["supported_D"] = 0
    individuals_money["supported_R"] = 0
    individuals_money["supported_OTHER"] = 0
    return individuals_money

#### Step 4
##### Get some more information about each bills

ProPublica provides an exhausting interface for different information about the US government.  Fascinating is the fact that they show which cosponsors support each bill. Therefore, I save which party supports which bill and which senator supports how many bills from each party.

Reference: *Congress API, propublica.org, visited: 24.02.2020, URL: https://projects.propublica.org/api-docs/congress-api/bills/#get-cosponsors-for-a-specific-bill*

In [12]:
# Import of interesting bill data 
individuals_money_TMP = calculateImportanceRepresentative()
progress = 0
errorbacklog = pd.DataFrame(["first"], columns=list('h'))

def getInterestingData(bill):
    global progress
    progress += 1
    if progress % 50 == 0 or progress == 1:
        print(progress)
    result = bill
    url = "https://api.propublica.org/congress/v1/115/bills/"+str(bill["bill_type"])+str(bill["bill_number"])+"/cosponsors.json"
    try:
        r = requests.get(url, headers = {"X-API-Key": "hGZo7tcTCDpenT7PvpVj2FudUvsMQG6w9nURbIGZ"})
        r.raise_for_status()
        if len(r.json()["results"]):
            if len(r.json()["results"]) > 0:
                for s in r.json()["results"][0]["cosponsors_by_party"]:  
                    result["cosponsors_by_party_"+s["party"]["id"]] = s["party"]["sponsors"]
                result["total_cosponser"] =  r.json()["results"][0]["number_of_cosponsors"]
                for s in r.json()["results"][0]["cosponsors"]:  
                    addVoteToRepresentative(s["cosponsor_id"], r.json()["results"][0]["sponsor_party"])
            else:
                print("No results for "+str(bill["bill_type"])+str(bill["bill_number"]))
        return result
    except requests.exceptions.HTTPError as err:
        errorbacklog = errorbacklog.append(pd.DataFrame([str(bill["bill_type"])+str(bill["bill_number"])], columns=list('h')))
        print(errorbacklog)
        print(err)
        
def addVoteToRepresentative(bioguide_id, party):
    switcher = {
        "D": "supported_D",
        "R": "supported_R",
    }
    pa = switcher.get(party, "supported_OTHER")
    individuals_money_tmp.loc[individuals_money["bioguide_id"] == bioguide_id, pa] += 1
    
### Comment this in if you want to reload all data from the external sources ###
#bills_better = bills[:]
#bills_better = bills_better.apply(lambda bill : getInterestingData(bill), axis=1) 
#print("done")
#bills_better.to_csv("./better_bills.csv")
#individuals_money_tmp.to_csv("./individuals_money.csv")
#individuals_money = individuals_money_tmp

#### Step 5
##### Calculate importance of each bill

This step evaluates the importance of each bill. Details can be found in my submission.

General rules:
The more opposite party member supported the bill the more important (+10%)
The more same party members supported the bill the more important (-10%)

In [13]:
# Bill importance
bills_in_better_tmp = pd.merge(bills_in_better,members[['party']],left_on=["sponsor_SGID"], right_index=True, how = 'left')
# Adjust naming
bills_in_better_tmp.loc[bills_in_better_tmp["party"] == "Republican", "party"] = "R"
bills_in_better_tmp.loc[bills_in_better_tmp["party"] == "Democrat", "party"] = "D"
bills_in_better_tmp["cosponsors_by_party_R"] = bills_in_better_tmp["cosponsors_by_party_R"].fillna(0.)
bills_in_better_tmp["cosponsors_by_party_D"] = bills_in_better_tmp["cosponsors_by_party_D"].fillna(0.)

# Normalize
bills_in_better_tmp = normalize(bills_in_better_tmp, "total_cosponser")

def setImportanceBill(bill):
    result = bill
    if bill["party"] == "R" or bill["party"] == "D":
        own_party_votes = bill["cosponsors_by_party_" + str(bill["party"])]
    else:
        own_party_votes = bill["total_cosponser"] - (bill["cosponsors_by_party_R"] + bill["cosponsors_by_party_D"])
    if bill["party"] == "R":
        other_party_votes = bill["cosponsors_by_party_D"]
    else:
        if bill["party"] == "D":
            other_party_votes = bill["cosponsors_by_party_R"]
        else:
            other_party_votes = (bill["cosponsors_by_party_R"] + bill["cosponsors_by_party_D"]) - bill["total_cosponser"]
    
    result["BillImportance"] =  1.1 * (float(other_party_votes)/float(bill["total_cosponser"]))
    result["BillImportance"] += 0.9 * (float(own_party_votes)  /float(bill["total_cosponser"]))
    result["BillImportance"] *= bill["total_cosponser_normalized"]
    return result

bills_in_better_tmp = bills_in_better_tmp.apply(lambda bill : setImportanceBill(bill), axis=1) 
bills_in_better_final = bills_in_better_tmp


In [14]:
ten_most_important_bills = bills_in_better_final.nlargest(10, "BillImportance")
print('The ten most importance bills: ')
ten_most_important_bills

The ten most importance bills: 


,bill_id,bill_number,bill_type,congress,cosponsors_by_party_D,cosponsors_by_party_R,introduced_at,sponsor_SGID,total_cosponser,party,total_cosponser_normalized,BillImportance
4664,hr2519-115,2519,hr,115,179.0,206.0,2017-05-18,1583,385,D,1.000000,1.007013
5050,hr930-115,930,hr,115,191.0,194.0,2017-02-07,1549,385,R,1.000000,0.999221
6699,hr820-115,820,hr,115,191.0,180.0,2017-02-02,1543,371,R,0.963542,0.966399
4932,hr299-115,299,hr,115,175.0,155.0,2017-01-05,1795,330,R,0.856771,0.861963
5589,hr392-115,392,hr,115,176.0,153.0,2017-01-10,1671,329,R,0.854167,0.860138
3042,hr5499-115,5499,hr,115,98.0,213.0,2018-04-12,1726,311,D,0.807292,0.837143
1468,hr1698-115,1698,hr,115,133.0,190.0,2017-03-23,1002,323,R,0.838542,0.823744
3455,hr2358-115,2358,hr,115,176.0,130.0,2017-05-04,1002,306,R,0.794271,0.806211
5132,hr2315-115,2315,hr,115,97.0,201.0,2017-05-03,1449,298,D,0.773438,0.800430
2392,hr3274-115,3274,hr,115,159.0,148.0,2017-07-17,1559,307,R,0.796875,0.799730


#### Step 6
##### Combine all evaluated data

Merge the data sets to map bill importance and the information about the members to the members of the House of Representatives.


In [15]:
members_of_115_all = pd.merge(members.reset_index(),individuals_money[["115_signatures_received","115_signatures_given","bioguide_id","AMOUNT",'AMOUNT_normalized', '115_signatures_received_normalized','115_signatures_given_normalized','in_congress_last_five_periods','importance']],left_on=["bioguide_id"], right_on = ['bioguide_id'], how = 'left').set_index('SGID')
print("Representatives of the 10 most important bills:")
members_of_115_all.loc[ten_most_important_bills["sponsor_SGID"]]

Representatives of the 10 most important bills:


,name,party,state,thomas_id,bioguide_id,congress,number_of_bills,115_signatures_received,115_signatures_given,AMOUNT,AMOUNT_normalized,115_signatures_received_normalized,115_signatures_given_normalized,in_congress_last_five_periods,importance
SGID,,,,,,,,,,,,,,,
1583,"Walz, Timothy J.",Democrat,MN,1856.0,W000799,115,11,514.0,350.0,7690306.11,0.447314,0.304322,0.253275,1.00,0.723657
1549,"Reichert, David G.",Republican,WA,1810.0,R000578,115,22,1161.0,145.0,7412045.22,0.430909,0.687389,0.104076,1.00,0.715455
1543,"McCaul, Michael T.",Republican,TX,1804.0,M001157,115,30,944.0,274.0,9744351.00,0.568406,0.558911,0.197962,1.00,0.784203
1795,"Valadao, David G.",Republican,CA,2105.0,V000129,115,5,380.0,281.0,7641944.22,0.444462,0.224985,0.203057,0.50,0.472231
1671,"Chaffetz, Jason",Republican,UT,1956.0,C001076,115,16,450.0,52.0,2518408.83,0.142415,0.266430,0.036390,1.00,0.571207
1726,"Richmond, Cedric L.",Democrat,LA,2023.0,R000588,115,11,676.0,294.0,9571708.74,0.558228,0.400237,0.212518,0.75,0.654114
1002,"Royce, Edward R.",Republican,CA,998.0,R000487,115,33,1685.0,212.0,7340372.91,0.426684,0.997632,0.152838,1.00,0.713342
1002,"Royce, Edward R.",Republican,CA,998.0,R000487,115,33,1685.0,212.0,7340372.91,0.426684,0.997632,0.152838,1.00,0.713342
1449,"Lynch, Stephen F.",Democrat,MA,1686.0,L000562,115,21,816.0,350.0,10387574.67,0.606325,0.483126,0.253275,1.00,0.803163


In [16]:
# Group bills by sponsor
bills_grouped_by_sponsor = bills_in_better_final.drop(columns="bill_id")
bills_grouped_by_sponsor = bills_grouped_by_sponsor.drop(columns="bill_number")
bills_grouped_by_sponsor = bills_grouped_by_sponsor.drop(columns="bill_type")
bills_grouped_by_sponsor = bills_grouped_by_sponsor.drop(columns="congress")
bills_grouped_by_sponsor = bills_grouped_by_sponsor.drop(columns="introduced_at")
bills_grouped_by_sponsor = bills_grouped_by_sponsor.drop(columns="total_cosponser_normalized")
bills_grouped_by_sponsor = bills_grouped_by_sponsor.groupby(['sponsor_SGID']).sum()
bills_grouped_by_sponsor = normalize(bills_grouped_by_sponsor, "BillImportance")

In [17]:
# Combine all members and bills with most interesting informationen
members_with_all_information = pd.merge(members_of_115_all.reset_index(), bills_grouped_by_sponsor, left_on=["SGID"], right_index=True, how = 'left').set_index('SGID')
# Filter members with no own bill or 0 own importance
deleteindex = members_with_all_information[(pd.notna(members_with_all_information["BillImportance"])) & members_with_all_information["importance"] == 0.].index
members_with_all_information = members_with_all_information.drop(deleteindex)
# Calculate overall Importance
def calculateOverAllImportance(m):
    result = m
    result["overallImportance"] = 0.5 * m["importance"] + 0.5 * m ["BillImportance_normalized"]
    return result

members_with_all_information = members_with_all_information.apply(lambda bill : calculateOverAllImportance(bill), axis=1) 
members_with_all_information.nlargest(10, "overallImportance")

,name,party,state,thomas_id,bioguide_id,congress,number_of_bills,115_signatures_received,115_signatures_given,AMOUNT,...,115_signatures_received_normalized,115_signatures_given_normalized,in_congress_last_five_periods,importance,cosponsors_by_party_D,cosponsors_by_party_R,total_cosponser,BillImportance,BillImportance_normalized,overallImportance
SGID,,,,,,,,,,,,,,,,,,,,,
1002,"Royce, Edward R.",Republican,CA,998.0,R000487,115,33,1685.0,212.0,7340372.91,...,0.997632,0.152838,1.00,0.713342,822.0,864.0,1686.0,4.292423,1.000000,0.856671
1758,"Cicilline, David N.",Democrat,RI,2055.0,C001084,115,50,1689.0,674.0,10138790.22,...,1.000000,0.489083,0.75,0.670829,1641.0,47.0,1689.0,3.862105,0.899749,0.785289
733,"Maloney, Carolyn B.",Democrat,NY,729.0,M000087,115,41,1329.0,435.0,11664144.51,...,0.786856,0.315138,1.00,0.840792,1195.0,136.0,1331.0,3.092045,0.720350,0.780571
284,"DeLauro, Rosa L.",Democrat,CT,281.0,D000216,115,38,1338.0,409.0,10629422.22,...,0.792185,0.296215,1.00,0.810292,1307.0,31.0,1338.0,3.061348,0.713198,0.761745
347,"Engel, Eliot L.",Democrat,NY,344.0,E000179,115,42,1289.0,500.0,8936514.00,...,0.763173,0.362445,1.00,0.760391,956.0,333.0,1289.0,3.090439,0.719975,0.740183
1646,"Paulsen, Erik",Republican,MN,1930.0,P000594,115,39,1328.0,203.0,7100318.85,...,0.786264,0.146288,1.00,0.706266,563.0,766.0,1328.0,3.305533,0.770086,0.738176
1319,"Lee, Barbara",Democrat,CA,1501.0,L000551,115,45,1217.0,751.0,10749907.02,...,0.720545,0.545124,1.00,0.813843,1178.0,39.0,1217.0,2.765735,0.644330,0.729086
1191,"Velazquez, Nydia M.",Democrat,NY,1184.0,V000081,115,43,1129.0,584.0,10383839.10,...,0.668443,0.423581,1.00,0.803053,949.0,186.0,1135.0,2.653293,0.618134,0.710593
1549,"Reichert, David G.",Republican,WA,1810.0,R000578,115,22,1161.0,145.0,7412045.22,...,0.687389,0.104076,1.00,0.715455,655.0,515.0,1163.0,3.024453,0.704603,0.710029
